<a href="https://colab.research.google.com/github/byte-kraken/MicroBit_TicTacToe/blob/master/DIN_to_ASME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# @title
# DIN to ASME Converter with Fluid Consideration (Water vs. Steam)
# Date: March 07, 2025
# Fluids: Natural Gas, Mixed Gas, Water, Steam, Combustion Air, Compressed Air, Exhaust Gas
# Note: Ratings for carbon steel (ASTM A105) based on ASME B16.5

# Mapping of DN (DIN) to NPS (ASME) sizes (in inches)
dn_to_nps = {
    15: 0.5, 20: 0.75, 25: 1, 32: 1.25, 40: 1.5, 50: 2, 65: 2.5, 80: 3,
    100: 4, 125: 5, 150: 6, 200: 8, 250: 10, 300: 12, 350: 14, 400: 16
}

# ASME B16.5 Pressure Ratings for ASTM A105 (carbon steel) in bar at various temperatures (°C)
asme_class_ratings = {
    150: {38: 19.6, 100: 17.7, 200: 15.8, 300: 13.8, 400: 11.0},
    300: {38: 51.1, 100: 46.6, 200: 45.1, 300: 41.1, 400: 34.8},
    400: {38: 68.2, 100: 62.1, 200: 60.2, 300: 54.8, 400: 46.4},
    600: {38: 102.1, 100: 93.2, 200: 90.2, 300: 82.2, 400: 69.6},
    900: {38: 153.2, 100: 139.8, 200: 135.3, 300: 123.3, 400: 104.4},
    1500: {38: 255.3, 100: 233.0, 200: 225.5, 300: 205.5, 400: 174.0},
    2500: {38: 425.5, 100: 388.3, 200: 375.8, 300: 342.5, 400: 290.0}
}

# Project-specific fluids
valid_fluids = [
    "Natural Gas", "Mixed Gas", "Water", "Steam",
    "Combustion Air", "Compressed Air", "Exhaust Gas"
]

# Fluid properties for consideration
fluid_notes = {
    "Natural Gas": "Flammable gas; 1.5x safety factor applied.",
    "Mixed Gas": "Flammable gas; 1.5x safety factor applied.",
    "Water": "Liquid; consider water hammer protection if dynamic flow.",
    "Steam": "High-temperature vapor; use stainless steel (e.g., A182 F304) if >400°C.",
    "Combustion Air": "Non-corrosive gas; standard carbon steel suitable.",
    "Compressed Air": "Non-corrosive gas; standard carbon steel suitable.",
    "Exhaust Gas": "Potentially corrosive/high-temp; consider stainless steel if >400°C."
}

def dn_to_nps_converter(dn_size):
    """Convert DN size to NPS size."""
    return dn_to_nps.get(dn_size, "DN size not found in mapping")

def interpolate_rating(class_rating, temp, temp_low, temp_high, rating_low, rating_high):
    """Linear interpolation for pressure rating between two temperatures."""
    if temp_low == temp_high:
        return rating_low
    return rating_low + (rating_high - rating_low) * (temp - temp_low) / (temp_high - temp_low)

def get_rating_at_temp(class_rating, temp):
    """Get the pressure rating for a class at a specific temperature."""
    temps = sorted(asme_class_ratings[class_rating].keys())
    if temp <= temps[0]:
        return asme_class_ratings[class_rating][temps[0]]
    if temp >= temps[-1]:
        return asme_class_ratings[class_rating][temps[-1]]
    for i in range(len(temps) - 1):
        if temps[i] <= temp <= temps[i + 1]:
            return interpolate_rating(
                class_rating, temp, temps[i], temps[i + 1],
                asme_class_ratings[class_rating][temps[i]],
                asme_class_ratings[class_rating][temps[i + 1]]
            )
    return None

def get_applicable_classes(pressure_bar, temp):
    """Return a list of ASME classes that can handle the given pressure at temperature."""
    applicable_classes = []
    for class_rating in asme_class_ratings:
        max_pressure = get_rating_at_temp(class_rating, temp)
        if pressure_bar <= max_pressure:
            applicable_classes.append((class_rating, max_pressure))
    return applicable_classes if applicable_classes else [("N/A", "Pressure exceeds available ratings")]

def suggest_minimum_class(pressure_bar, temp):
    """Suggest the minimum ASME class rating for a given pressure and temperature."""
    for class_rating in sorted(asme_class_ratings.keys()):
        max_pressure = get_rating_at_temp(class_rating, temp)
        if pressure_bar <= max_pressure:
            return class_rating
    return "Pressure exceeds available class ratings"

def get_fluid_code(fluid):
    """Generate a 2-letter code for the fluid for P&ID tagging."""
    fluid_map = {
        "Natural Gas": "NG", "Mixed Gas": "MG", "Water": "WA",
        "Steam": "ST", "Combustion Air": "CA", "Compressed Air": "CP",
        "Exhaust Gas": "EG"
    }
    return fluid_map.get(fluid, "XX")

def adjust_pressure_for_fluid(pressure_bar, fluid, temp):
    """Adjust pressure with safety factor or fluid-specific logic."""
    if fluid in ["Natural Gas", "Mixed Gas"]:  # Flammable gases
        return pressure_bar * 1.5, "1.5x safety factor applied for flammable gas"
    elif fluid == "Water" and temp > 100:
        return pressure_bar, "Warning: Water above 100°C implies pressure vessel; verify conditions"
    elif fluid == "Steam" and temp > 400:
        return pressure_bar, "Warning: Steam >400°C; consider stainless steel (e.g., A182 F304)"
    return pressure_bar, ""

def main():
    print("DIN to ASME Converter with Fluid Consideration (Water vs. Steam)")
    print("==============================================================")
    print("Valid fluids:", ", ".join(valid_fluids))

    # User input
    try:
        dn_size = int(input("\nEnter DN size (e.g., 250 for DN250): "))
        pressure_bar = float(input("Enter operating pressure (bar): "))
        temp_c = float(input("Enter operating temperature (°C): "))
        fluid = input("Enter fluid type (from list above): ").strip()
    except ValueError:
        print("Invalid input. Please enter numeric values for DN size, pressure, and temperature.")
        return

    # Validate fluid
    if fluid not in valid_fluids:
        print(f"Error: '{fluid}' is not a valid fluid. Choose from: {', '.join(valid_fluids)}")
        return

    # Adjust pressure for fluid-specific considerations
    adjusted_pressure, fluid_warning = adjust_pressure_for_fluid(pressure_bar, fluid, temp_c)
    if adjusted_pressure != pressure_bar or fluid_warning:
        print(f"\nAdjusted pressure: {adjusted_pressure:.1f} bar ({fluid_warning})")

    # Convert DN to NPS
    nps_size = dn_to_nps_converter(dn_size)
    if isinstance(nps_size, str):
        print(nps_size)
        return
    else:
        print(f"DN{dn_size} converts to NPS {nps_size}\"")

    # Fluid and temperature information
    print(f"Fluid specified: {fluid}")
    print(f"Operating temperature: {temp_c}°C")
    print(f"Fluid note: {fluid_notes[fluid]}")

    # Suggest minimum ASME class based on adjusted pressure
    min_class = suggest_minimum_class(adjusted_pressure, temp_c)
    if isinstance(min_class, int):
        min_rating = get_rating_at_temp(min_class, temp_c)
        print(f"Minimum ASME Class for {adjusted_pressure:.1f} bar at {temp_c}°C: Class {min_class}")
        print(f"Maximum allowable pressure for Class {min_class}: {min_rating:.1f} bar")
    else:
        print(min_class)
        return

    # Predict all applicable ASME classes
    applicable_classes = get_applicable_classes(adjusted_pressure, temp_c)
    print("\nPredicted Applicable ASME Classes and Ratings:")
    print("---------------------------------------------")
    print("Class\tMax Pressure (bar)")
    for class_rating, max_pressure in applicable_classes:
        print(f"{class_rating}\t{max_pressure:.1f}")




if __name__ == "__main__":
    main()

DIN to ASME Converter with Fluid Consideration (Water vs. Steam)
Valid fluids: Natural Gas, Mixed Gas, Water, Steam, Combustion Air, Compressed Air, Exhaust Gas


KeyboardInterrupt: Interrupted by user